In [34]:
import altair as alt
import os
import pandas as pd
import glob
import tensorflow as tf

In [35]:
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [36]:
summary_iterator = tf.compat.v1.train.summary_iterator

def get_section_results(file, xtag='step', ytag='train_mse_loss'):
    """
        requires tensorflow==1.12.0
    """

    X = []
    Y = []
    for e in summary_iterator(file):
        for v in e.summary.value:
            if v.tag == xtag:
                X.append(v.simple_value)
            elif v.tag == ytag:
                Y.append(v.simple_value)
    return X, Y


def make_dataframe(eventfiles, xtag='step', ytag='train_mse_loss'):
    dfs = []
    for eventfile in eventfiles:
        dirname = os.path.basename(os.path.dirname(eventfile))
        X, Y = get_section_results(eventfile, xtag, ytag)
        n = len(X)
        msg = f"{(len(X), len(Y))}"
        length = min(len(X), len(Y))
        X = X[:length]
        Y = Y[:length]
        assert len(X) == len(Y), msg
        log_df = pd.DataFrame.from_dict({xtag: X, ytag: Y, 'exp': [dirname] * length})
        dfs.append(log_df)
    
    return pd.concat(dfs)

In [37]:
version = 206 # Try 206, which ran for longer

logdir = f'/home/ubuntu/deeponet-fno/lightning_logs/version_{version}/events*'
eventfiles = glob.glob(logdir)
xtag = 'epoch'
ytag = 'val_mse_loss'
df_soft  = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)
df_soft['step'] = df_soft.index + 1
df_soft['model'] = "Soft constrained"

In [38]:
version = 286
logdir = f'/home/ubuntu/deeponet-fno/lightning_logs/version_{version}/events*'
eventfiles = glob.glob(logdir)
xtag = 'epoch'
ytag = 'val_mse_loss'
df_hard  = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)
df_hard['step'] = df_hard.index + 1
df_hard['model'] = "Hard constrained (ours)"


In [39]:
source = pd.concat((df_soft, df_hard), 0)

/tmp/ipykernel_727992/2290452603.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  source = pd.concat((df_soft, df_hard), 0)


In [40]:
alt.Chart(source).mark_line().encode(
    x=alt.X('step', scale=alt.Scale(type='log')),
    y=alt.Y('val_mse_loss', scale=alt.Scale(type='log')),
    color='model',
    strokeDash='model',
)

/opt/conda/envs/iclr-reb/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)